In [ ]:
import pandas as pd
import gzip
import os
import requests
import time
import io
import missingno as msno

# Récupération et traitement du fichier DVF

In [ ]:
# Liste des années à télécharger
years = [2017, 2018, 2019, 2020, 2021, 2022]

In [ ]:
def process_dvf(years, base_url='https://files.data.gouv.fr/geo-dvf/latest/csv/', output_file='full_dvf.csv'):
    df_list = []

    for year in years:
        start_time = time.time()

        # Étape 1: Télécharger le fichier
        url = f'{base_url}{year}/full.csv.gz'
        response = requests.get(url)
        
        if response.status_code == 200:
            decompressed_file = gzip.decompress(response.content)
            df = pd.read_csv(io.StringIO(decompressed_file.decode('utf-8')))
            print(f"Étape 1 ({year}): Téléchargement du fichier terminé - Temps écoulé: {time.time() - start_time:.2f} secondes")

            # Étape 2: Filtrer les lignes avec la valeur 'Vente' dans la colonne 'nature_mutation'
            start_time = time.time()
            df = df[df['nature_mutation'] == 'Vente']
            print(f"Étape 2 ({year}): Filtrage 'Vente' terminé - Temps écoulé: {time.time() - start_time:.2f} secondes")

            # Étape 3: Supprimer les lignes avec une valeur null dans la colonne 'surface_reelle_bati'
            start_time = time.time()
            df = df.dropna(subset=['surface_reelle_bati'])
            print(f"Étape 3 ({year}): Suppression des valeurs null dans 'surface_reelle_bati' terminée - Temps écoulé: {time.time() - start_time:.2f} secondes")

            # Étape 4: Supprimer les lignes avec une valeur null dans la colonne 'valeur_fonciere'
            start_time = time.time()
            df = df.dropna(subset=['valeur_fonciere'])
            print(f"Étape 4 ({year}): Suppression des valeurs null dans 'valeur_fonciere' terminée - Temps écoulé: {time.time() - start_time:.2f} secondes")

            # Étape 5: Filtrer les lignes avec les valeurs 'Maison' ou 'Appartement' dans la colonne 'type_local'
            start_time = time.time()
            df = df[df['type_local'].isin(['Maison', 'Appartement'])]
            print(f"Étape 5 ({year}): Filtrage 'Maison' et 'Appartement' terminé - Temps écoulé: {time.time() - start_time:.2f} secondes")

            # Étape 6: Supprimer les lignes avec une valeur null dans les colonnes 'longitude' et 'latitude'
            start_time = time.time()
            df = df.dropna(subset=['longitude', 'latitude'])
            print(f"Étape 6 ({year}): Suppression des valeurs nulles dans 'longitude' et 'latitude' effectuée - Temps écoulé: {time.time() - start_time:.2f} secondes")
            
            # Ajouter le DataFrame à la liste
            df_list.append(df)
        else:
            print(f"Erreur lors du téléchargement du fichier pour l'année {year}")

    # Concaténer les DataFrames
    concatenated_df = pd.concat(df_list, ignore_index=True)

    # Enregistrer le DataFrame concaténé dans un fichier CSV
    concatenated_df.to_csv(output_file, index=False)
    print(f'Fichier CSV concaténé enregistré : {output_file}')


In [ ]:
# Appeler la fonction avec les années souhaitées
process_dvf(years)

# Analyse du fichier final

In [ ]:
full_dvf= pd.read_csv('full_dvf.csv')

In [ ]:
msno.bar(full_dvf)